In [1]:
import os
import time
import random
import numpy as np

import tensorflow as tf
import tensorflow.contrib.layers as layers

import gym
from gym import spaces
from PIL import Image
from collections import deque, namedtuple

from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

/root/miniconda3/envs/tf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from environments import Snake
from new_agents import *

In [3]:
env = Snake((8, 8))
#aa = DuelDQNAgent(env=env, num_actions=3, state_shape=[8, 8, 5], 
#              save_path="snake_models", model_name="test")
#aa = CatDQNAgent(env=env, num_actions=3, state_shape=[8, 8, 5], num_atoms=51,
#                 save_path="snake_models", model_name="test")
#aa = SACAgent(env=env, num_actions=3, temperature=0.05,
#              save_path="snake_models", model_name="test")
aa = GMMDQNAgent(env=env, num_actions=3, state_shape=[8, 8, 5],
                   save_path="snake_models", model_name="test_wasser", metric='wasserstein')

In [4]:
aa.set_parameters(max_episode_length=2000, replay_memory_size=100000, replay_start_size=10000,
                  discount_factor=0.999, final_eps=0.01, annealing_steps=100000)

In [ ]:
aa.train(gpu_id=5, exploration="boltzmann", save_freq=2000000, max_num_epochs=1000, tau=1e-3, batch_size=32)

frame count: 4790
average reward: -0.874
epsilon: 0.953
average lifetime: 9.58
-------------------------------
frame count: 11029
average reward: -0.844
epsilon: 0.891
average lifetime: 12.478
-------------------------------
frame count: 18393
average reward: -0.834
epsilon: 0.818
average lifetime: 14.728
-------------------------------
frame count: 28667
average reward: -0.742
epsilon: 0.716
average lifetime: 20.548
-------------------------------
frame count: 45422
average reward: -0.544
epsilon: 0.55
average lifetime: 33.51
-------------------------------
frame count: 89276
average reward: 0.47
epsilon: 0.116
average lifetime: 87.708
-------------------------------
frame count: 142898
average reward: 5.296
epsilon: 0.01
average lifetime: 107.244
-------------------------------
frame count: 202625
average reward: 10.258
epsilon: 0.01
average lifetime: 119.454
-------------------------------
frame count: 283109
average reward: 14.366
epsilon: 0.01
average lifetime: 160.968
-----------